# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

Данные имеют вид:  
1. id — уникальный идентификатор скважины;
2. f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
3. product — объём запасов в скважине (тыс. баррелей).


In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
from numpy.random import RandomState

## Загрузка и подготовка данных

In [2]:
data_geo_0 = pd.read_csv('/datasets/geo_data_0.csv')
data_geo_1 = pd.read_csv('/datasets/geo_data_1.csv')
data_geo_2 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_geo_0.head()

,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647


In [4]:
data_geo_0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [5]:
data_geo_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


In [6]:
data_geo_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB


Пропущенных значений нет. Проверим на дубликаты.

In [7]:
data_geo_0.duplicated().sum()

0

In [8]:
data_geo_1.duplicated().sum()

0

In [9]:
data_geo_2.duplicated().sum()

0

Удалим признак идентефикатора. Для обучения моделей он нам не понадобиться.

In [10]:
data_geo_0 = data_geo_0.drop(['id'], axis=1)
data_geo_0.head()

,f0,f1,f2,product
0,0.705745,-0.497823,1.221170,105.280062
1,1.334711,-0.340164,4.365080,73.037750
2,1.022732,0.151990,1.419926,85.265647
3,-0.032172,0.139033,2.978566,168.620776
4,1.988431,0.155413,4.751769,154.036647


In [11]:
data_geo_1 = data_geo_1.drop(['id'], axis=1)
data_geo_1.head()

,f0,f1,f2,product
0,-15.001348,-8.276000,-0.005876,3.179103
1,14.272088,-3.475083,0.999183,26.953261
2,6.263187,-5.948386,5.001160,134.766305
3,-13.081196,-11.506057,4.999415,137.945408
4,12.702195,-8.147433,5.004363,134.766305


In [12]:
data_geo_2 = data_geo_2.drop(['id'], axis=1)
data_geo_2.head()

,f0,f1,f2,product
0,-1.146987,0.963328,-0.828965,27.758673
1,0.262778,0.269839,-2.530187,56.069697
2,0.194587,0.289035,-5.586433,62.871910
3,2.236060,-0.553760,0.930038,114.572842
4,-0.515993,1.716266,5.899011,149.600746


При подготовке данных мы избавились от ненужного нам столбца идентефикаторов, а так же проверели данные на пропуски и дубликаты.

In [13]:
data_geo_0.corr()

,f0,f1,f2,product
f0,1.000000,-0.440723,-0.003153,0.143536
f1,-0.440723,1.000000,0.001724,-0.192356
f2,-0.003153,0.001724,1.000000,0.483663
product,0.143536,-0.192356,0.483663,1.000000


In [14]:
data_geo_1.corr()

,f0,f1,f2,product
f0,1.000000,0.182287,-0.001777,-0.030491
f1,0.182287,1.000000,-0.002595,-0.010155
f2,-0.001777,-0.002595,1.000000,0.999397
product,-0.030491,-0.010155,0.999397,1.000000


In [15]:
data_geo_2.corr()

,f0,f1,f2,product
f0,1.000000,0.000528,-0.000448,-0.001987
f1,0.000528,1.000000,0.000779,-0.001012
f2,-0.000448,0.000779,1.000000,0.445871
product,-0.001987,-0.001012,0.445871,1.000000


Из таблиц корреляции самое важное что можем отметить это очень сильную корреляцию между f2 и product во втором регионе. Скорее всего имеет смысл даже удалить столбец f2 из второго датафрейма. Во всех остальных регионах все отлично.

## Обучение и проверка модели

Напишем функцию для того, чтобы подготовить данные к обучению модели(разобьем данные на тестовую и валидационную выборку и произведем стандартизацию данных), а так же построим модель на основе этих данных и посчитаем RMSE.

In [16]:
def rmse_data(data):
    features = data.drop('product', axis=1)
    target = data['product']
    
    features_train, features_valid, target_train, target_valid = train_test_split(features, target, test_size=0.25, 
                                                                                  random_state=12345)
    
    scaler = StandardScaler()
    scaler.fit(features_train)
    features_train = pd.DataFrame(scaler.transform(features_train))
    features_valid = pd.DataFrame(scaler.transform(features_valid))
    target_train = target_train.reset_index(drop=True)
    target_valid = target_valid.reset_index(drop=True)
    
    model = LinearRegression()
    model.fit(features_train, target_train)
    predicted_valid = model.predict(features_valid)
    
    return mean_squared_error(target_valid, predicted_valid) ** 0.5, predicted_valid, target_valid

In [17]:
data_geo_0_rmse, data_geo_0_predicted, data_geo_0_target_valid = rmse_data(data_geo_0)
data_geo_1_rmse, data_geo_1_predicted, data_geo_1_target_valid = rmse_data(data_geo_1)
data_geo_2_rmse, data_geo_2_predicted, data_geo_2_target_valid = rmse_data(data_geo_2)

In [18]:
data_geo_0_predicted = pd.Series(data_geo_0_predicted)
data_geo_1_predicted = pd.Series(data_geo_1_predicted)
data_geo_2_predicted = pd.Series(data_geo_2_predicted)

In [19]:
print('RMSE в первом регионе:', data_geo_0_rmse)
print('Средний запас предсказанного сырья в первом регионе:', data_geo_0_predicted.mean())

RMSE в первом регионе: 37.5794217150813
Средний запас предсказанного сырья в первом регионе: 92.59256778438035


In [20]:
print('RMSE во втором регионе:', data_geo_1_rmse)
print('Средний запас предсказанного сырья во втором регионе:', data_geo_1_predicted.mean())

RMSE во втором регионе: 0.893099286775617
Средний запас предсказанного сырья во втором регионе: 68.728546895446


In [21]:
print('RMSE в третьем регионе:', data_geo_2_rmse)
print('Средний запас предсказанного сырья в третьем регионе:', data_geo_2_predicted.mean())

RMSE в третьем регионе: 40.02970873393434
Средний запас предсказанного сырья в третьем регионе: 94.96504596800489


RMSE в первом регионе ниже, чем в третьем, средний запас предсказанного сырья так же ниже, чем в третьем.  
RMSE во втором регионе самое маленькое, однако и средний запас сырья в этом регионе ниже всего.  
RMSE в третьем регионе самое высокое, и в тоже время и средний запас сырья выше, чем в других регионах.

## Подготовка к расчёту прибыли

Создадим нужные нам константы

In [22]:
best_points = 200
total_points = 500
budget = 10**10
product_price = 450000
quant = .025

Посчитаем минимальное количество тыс. барралей для безубыточности.

In [23]:
min_barrels = budget / product_price / best_points
min_barrels

111.11111111111111

Минимальное количство тысяч барралей для безубыточности проекта 111.11111111111111. Во всех наших трех регионах средний запас гораздо ниже. Ближе всего по запасу третий регион, где средний запас равен 94.96504596800489.

In [24]:
print('Истинный средний запас первого региона равен', data_geo_0_target_valid.mean())
print('Предсказанный средний запас первого регона', data_geo_0_predicted.mean())

Истинный средний запас первого региона равен 92.07859674082927
Предсказанный средний запас первого регона 92.59256778438035


In [25]:
print('Истинный средний запас второго региона равен', data_geo_1_target_valid.mean())
print('Предсказанный средний запас второго регона', data_geo_1_predicted.mean())

Истинный средний запас второго региона равен 68.72313602435997
Предсказанный средний запас второго регона 68.728546895446


In [26]:
print('Истинный средний запас третьего региона равен', data_geo_2_target_valid.mean())
print('Предсказанный средний запас третьего регона', data_geo_2_predicted.mean())

Истинный средний запас третьего региона равен 94.88423280885438
Предсказанный средний запас третьего регона 94.96504596800489


Истинные и предсказанные средние очень близки друг к другу, однако RMSE второго региона гораздо ниже остальных. Изначально MSE это сумма квадратов ошибок объектов / кол-во объектов. Соотвественно средние значения и RMSE могут быть вообще не связаны, например у нас есть очень сильные ошибки в плюс и очень сильные ошибки в минус, при подсчете среднего они будут друг дурга компенсировать, а сумма квадратов ошибок при этом будет расти.

Рассмотрим предсказания только для лучших 200 шахт каждого региона. Возьмем среднее для них и сравним с минимальным количеством тысяч барралей.

In [27]:
data_geo_0_predicted.sort_values(ascending=False).head(best_points).mean()

155.511654194057

In [28]:
data_geo_1_predicted.sort_values(ascending=False).head(best_points).mean()

138.73013391081716

In [29]:
data_geo_2_predicted.sort_values(ascending=False).head(best_points).mean()

148.01949329159174

Для топ 200 шахт каждого региона, регион с самым большим средним предсказанным оказался первый, за ним идет третий и худший по среднему предсказанному - второй регион. В целом для топ 200 шахт все 3 региона оказались прибыльными, так как везде среднее выше порога для безубыточности.

In [30]:
def profit(target, predictions):
    preds_sorted = predictions.sort_values(ascending=False)[:best_points]
    selected = target[preds_sorted.index]
    return selected.sum() * product_price - budget

In [31]:
print('Прибыль 200 лучших шахт первого региона составила:', profit(data_geo_0_target_valid, data_geo_0_predicted))
print('Прибыль 200 лучших шахт второго региона составила:', profit(data_geo_1_target_valid, data_geo_1_predicted))
print('Прибыль 200 лучших шахт третьего региона составила:', profit(data_geo_2_target_valid, data_geo_2_predicted))

Прибыль 200 лучших шахт первого региона составила: 3320826043.1398506
Прибыль 200 лучших шахт второго региона составила: 2415086696.681511
Прибыль 200 лучших шахт третьего региона составила: 2710349963.5998325


## Расчёт прибыли и рисков 

In [32]:
state = np.random.RandomState(12345)

In [33]:
def risk_and_profit(target, predictions):
    values = []
    for i in range(0, 1000):
        sample = predictions.sample(n=total_points, replace=True, random_state=state)
        values.append(profit(target, sample))
    values = pd.Series(values)    
    print('Средняя выручка в регионе', values.mean())
    lower = values.quantile(quant)
    upper = values.quantile(1-quant)
    print('2.5%-квантиль:', lower)
    print('97.5%-квантиль:', upper)
    print('Вероятость убытка:', (values.values<0).mean()*100, '%')

In [34]:
risk_and_profit(data_geo_0_target_valid, data_geo_0_predicted)

Средняя выручка в регионе 396164984.8023711
2.5%-квантиль: -111215545.89049526
97.5%-квантиль: 909766941.5534226
Вероятость убытка: 6.9 %


In [35]:
risk_and_profit(data_geo_1_target_valid, data_geo_1_predicted)

Средняя выручка в регионе 461155817.2772397
2.5%-квантиль: 78050810.7517417
97.5%-квантиль: 862952060.2637234
Вероятость убытка: 0.7000000000000001 %


In [36]:
risk_and_profit(data_geo_2_target_valid, data_geo_2_predicted)

Средняя выручка в регионе 392950475.17060447
2.5%-квантиль: -112227625.37857565
97.5%-квантиль: 934562914.5511636
Вероятость убытка: 6.5 %


После оценки рисков предлагается выбрать 2 регион, где по результатам исследования наибольшая средняя выручка(461155817.2772397), а так же самая маленькая веротяность убытка - 0.7%. В первом регионе вероятность убытка 6.9%, а во втором 6.5%, что противоречит условию того, что вероятность риска должна быть менее 2.5%.